In [2]:
from MQGO import data, main, ML_engine, optimizer, QC_engine 
from deepmd.calculator import DP
import subprocess
import numpy as np
import json

2022-05-16 09:47:51.038656: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-16 09:47:51.038794: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [10]:
d_steps = [6000]
tot_steps = [15000]
subprocess.run('cp CdSe_2.xyz CdSe.xyz', shell=True)

index = 0
for ii in range(0,len(d_steps)):
    for jj in range(0,len(tot_steps)):
        js_setting = {}
        with open('./input0.json', 'r') as file:
            json_setting = json.load(file)
            
        json_setting['learning_rate']['decay_steps'] = d_steps[ii]
        json_setting['learning_rate']['start_lr'] = 0.01
        json_setting['training']['numb_steps'] = tot_steps[jj]

        with open('./input.json', 'w') as js:
            json.dump(json_setting, js, indent=4)
        
        setting = {'basis':'def2-tzvp', 'ecp':'lanl2dz', 'spin':0, 'xc':'pbe', 'restricted':True, 'verbose':3, 'max_cycle':500}
        geo = main.MLgeomopt(qc_engine='pyscf', ml_engine='deepmd', work_path='./', xyz_path='./CdSe.xyz', consistensy_tol=0.01, **setting)
        geo.opt_algorithm = 'lbfgs'
        geo.debug = True
        geo.have_init_qc_data = False
        geo.kernel()
        
        Opt = optimizer.Optimizer(xyz_path="CdSe_2.xyz",
									work_path="./",
									ml_engine='deepmd',
									outter_cycle=1,
									algorithm='pyberny',
									max_opt_cycle=500)
        atom0 = Opt.ase_read_xyz()
        atom0.calc = DP(model='graph.pb')
        print(atom0.get_positions())
        force0_ml = atom0.get_forces()

        forces = np.loadtxt('raw/force.raw')
        force0_qc = forces[0].reshape((6,3))

        delta = force0_qc-force0_ml
        with open('debug_diff_force.txt', 'a') as file:
            file.write('%24.18f\n'%atom0.get_potential_energy())
            for i in range(0, len(force0_qc)):
                for j in range(0, 3):
                    file.write('%24.18f'%delta[i][j])
                file.write('\n')
            file.write('\n')

        subprocess.run('cp CdSe_2.xyz CdSe.xyz', shell=True)

converged SCF energy = -114.913245070908
--------------- RKS gradients ---------------
         x                y                z
0 Se     0.0146178367    -0.0000000000     0.0000000000
1 Se    -0.0146178375    -0.0000000000     0.0000000000
2 Cd     0.0000000072     0.4383880410     0.0000000000
3 Cd     0.0000000072    -0.4383880410     0.0000000000
----------------------------------------------
--------------- RKS gradients ---------------
         x                y                z
0 Se     0.0146178367    -0.0000000000     0.0000000000
1 Se    -0.0146178375    -0.0000000000     0.0000000000
2 Cd     0.0000000072     0.4383880410     0.0000000000
3 Cd     0.0000000072    -0.4383880410     0.0000000000
----------------------------------------------
converged SCF energy = -114.918531116402
--------------- RKS gradients ---------------
         x                y                z
0 Se     0.0244807623    -0.0000000000     0.0000000000
1 Se    -0.0244807626    -0.0000000000     0.00

2022-05-16 14:43:59.133913: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-16 14:43:59.134053: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term
DEEPMD INFO    Calculate neighbor statistics... (add --skip-neighbor-stat to skip this step)
2022-05-16 14:44:04.399076: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-16 14:44:04.411183: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed cal

optimizer.py 77: [[-2.0118679  0.         0.       ]
 [ 2.0118679  0.         0.       ]
 [ 0.         1.7205108  0.       ]
 [ 0.        -1.7205108  0.       ]]
       Step     Time          Energy          fmax
LBFGS:    0 14:50:24    -3125.195023   4.26627e+00
LBFGS:    1 14:50:24    -3125.672826   3.43944e+00
LBFGS:    2 14:50:24    -3127.071438   4.51320e+00
LBFGS:    3 14:50:24    -3125.696269   3.24708e+00
LBFGS:    4 14:50:24    -3123.831663   8.23659e+00
LBFGS:    5 14:50:24    -3125.686484   3.33180e+00
LBFGS:    6 14:50:24    -3126.968056   5.45714e+00
LBFGS:    7 14:50:24    -3125.460555   3.37115e+00
LBFGS:    8 14:50:24    -3124.676110   4.01018e+00
LBFGS:    9 14:50:24    -3124.642000   2.50889e+00
LBFGS:   10 14:50:24    -3124.768217   1.40368e+00
LBFGS:   11 14:50:24    -3124.759498   3.81121e-01
LBFGS:   12 14:50:24    -3124.759276   9.27676e-01
LBFGS:   13 14:50:24    -3124.757336   6.06774e-01
LBFGS:   14 14:50:24    -3124.758350   8.51599e-01
LBFGS:   15 14:50:24  

AssertionError: SCF is not converged, please modify related paramaters and rerun the calculations.

In [ ]:
a,b,c = data.read_xyz('CdSe_2.xyz',index=1,output='regular')
print(b)

In [ ]:
mo = np.loadtxt('water.xyz',usecols=(1,2,3),skiprows=10)
# print(mo[1])
atoms =  ''
for i in range(0, len(b)):
    _mo = ''
    for j in range(0, 3):
        _mo += str(mo[i][j]) + '  '
    atoms += b[i] + '  ' + _mo + '\n'
print(atoms)

In [35]:
from pyscf import gto, scf, geomopt
from pyscf.data.nist import BOHR, HARTREE2EV
mol = gto.Mole(atom=atoms, basis='6-31g', spin=0).build()
mf = scf.RHF(mol)
mf.max_cycle = 500
mf.kernel()
print(mf.e_tot * HARTREE2EV)
mol_eq = mf.Gradients().optimizer(solver='berny').kernel()

converged SCF energy = -150.91286083677
-4106.548111611891


In [3]:
from ase.io import read
from ase.optimize import LBFGS
a = read('./CdSe_2.xyz', index=':')[0]
a.calc=DP(model='graph-compress.pb')
opt=LBFGS(a)
opt.run()

       Step     Time          Energy          fmax
LBFGS:    0 17:41:03    -3126.814622         1.367256
LBFGS:    1 17:41:03    -3126.870969         1.515481
LBFGS:    2 17:41:03    -3126.580287         0.990453
LBFGS:    3 17:41:03    -3126.613426         0.931764
LBFGS:    4 17:41:03    -3126.785507         0.677808
LBFGS:    5 17:41:03    -3126.500476         1.278227
LBFGS:    6 17:41:03    -3126.360157         1.931378
LBFGS:    7 17:41:03    -3126.475493         1.869027
LBFGS:    8 17:41:03    -3126.921155         1.130549
LBFGS:    9 17:41:03    -3126.855139         0.581809
LBFGS:   10 17:41:03    -3126.954638         1.112587
LBFGS:   11 17:41:03    -3127.076942         1.564375
LBFGS:   12 17:41:03    -3126.948635         1.115520
LBFGS:   13 17:41:03    -3126.835143         0.515549
LBFGS:   14 17:41:03    -3126.889379         1.151211
LBFGS:   15 17:41:03    -3126.783298         0.687865
LBFGS:   16 17:41:03    -3126.756730         0.161381
LBFGS:   17 17:41:03    -3126.7

True

In [3]:
pwd

'/home/jingheng/ML_QC_GEOM_OPT/test/CdSe/2'